# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

### Структура данных:

- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

### План исследования

```
1) Изучить файл с данными.
2) Разбить данные на три выборки: обучающая, валидационная и тестовая.
3) Исследовать три модели классификации: Решающее дерево, Случайный лес и Логистическая регрессия.
4) Найти оптимальные параметры для каждой модели и выбрать одну из них для обучения модели.
5) Оценить точность обученной модели.
6) Оценить адекватность модели.
```

In [122]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore", category=RuntimeWarning)
pd.options.mode.chained_assignment = None 

## 1 - Откройте и изучите файл

In [123]:
df = pd.read_csv('/Users/yuliabezginova/Documents/DS/praktikum/project-5_ML_mobile-learning/users_behavior.csv')

In [124]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [125]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


### Вывод:

Перед нами стоит задача ___классификации___. За целевой признак берем столбец ___is_ultra___. Остальные признаки помогут нам предсказывать решение по столбцу _is_ultra_. В датасете 3200 наблюдений, это не так много, а значит, может возникнуть проблема _underfitting_: если модель будет хорошо работать на обучающем наборе, но не сможет обобщать результат на новые данные.

## 2 - Разбейте данные на выборки

Разбивать исходные датасет будем в два этапа.

```
Разобьем sample в следующей пропорции:
- 70% train_data
- 10% validation_data
- 20% test_data

```

In [126]:
train_data, other_data = train_test_split(df, test_size=0.30, random_state=2355555)
validation_data, test_data = train_test_split(other_data, test_size=0.70, random_state=2355555)

print('Размер тренериующей выборки', train_data.shape[0])
print('Размер валидационной выборки', validation_data.shape[0])
print('Размер тестовой выборки', test_data.shape[0])

Размер тренериующей выборки 2249
Размер валидационной выборки 289
Размер тестовой выборки 676


Выделим определяющие и целевой признаки модели.

In [127]:
train_data_features = train_data.drop(['is_ultra'], axis=1)
train_data_target = train_data['is_ultra']

In [128]:
validation_data_features = validation_data.drop(['is_ultra'], axis=1)
validation_data_target = validation_data['is_ultra']

In [129]:
test_data_features = test_data.drop(['is_ultra'], axis=1)
test_data_target = test_data['is_ultra']

**Данные были разбиты на три подвыборки в пропорции 70/10/30.**

- Обучение пройдет на данных train_data
- Валидация модели на данных validation_data
- Лучшая модель по валидации будет применена на данных test_data


## 3 - Исследуйте модели

### 3.1 Decision Tree

In [130]:
%%time

best_depth = 0
best_accuracy = 0

for depth in range(1, 101):
    model = DecisionTreeClassifier(random_state=2355555, max_depth=depth)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(validation_data_features)
    accuracy = accuracy_score(validation_data_target, valid_predictions)
#   print('Глубина дерева', depth,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_depth = depth
        best_accuracy = accuracy
print('Лучшая глубина дерева', best_depth,'Лучшая точность', best_accuracy)        

Лучшая глубина дерева 5 Лучшая точность 0.7785467128027682
CPU times: user 1.41 s, sys: 37.2 ms, total: 1.44 s
Wall time: 1.73 s


**_Decision Tree:_ лучший показатель при глубине дерева 5.**

Начинаем перебирать разные гиперпараметры модели.

- ___max_depth=8___

In [131]:
%%time

best_tree_model = DecisionTreeClassifier(random_state=2355555, max_depth=8)
best_tree_model.fit(train_data_features, train_data_target)
valid_predictions = best_tree_model.predict(validation_data_features)
accuracy = accuracy_score(validation_data_target, valid_predictions)
print('Глубина дерева:', 8,'\nТочность (max_depth=8):', accuracy)

Глубина дерева: 8 
Точность (max_depth=8): 0.7716262975778547
CPU times: user 15.9 ms, sys: 3.45 ms, total: 19.3 ms
Wall time: 27.7 ms


- ___criterion="gini" --> criterion="entropy"___

In [132]:
%%time

model = DecisionTreeClassifier(random_state=2355555, max_depth=8, criterion="entropy")
model.fit(train_data_features, train_data_target)
valid_predictions = best_tree_model.predict(validation_data_features)
accuracy = accuracy_score(validation_data_target, valid_predictions)
print('Глубина дерева:', 8,'\nТочность (max_depth=8, criterion="entropy"):', accuracy)

Глубина дерева: 8 
Точность (max_depth=8, criterion="entropy"): 0.7716262975778547
CPU times: user 14.8 ms, sys: 2.9 ms, total: 17.7 ms
Wall time: 35.2 ms


Точность модели не изменилась при criterion="entropy".

- ___splitter="best" --> splitter='random'___

In [133]:
%%time

model = DecisionTreeClassifier(random_state=2355555, max_depth=8, splitter="random")
model.fit(train_data_features, train_data_target)
valid_predictions = best_tree_model.predict(validation_data_features)
accuracy = accuracy_score(validation_data_target, valid_predictions)
print('Глубина дерева:', 8,'\nТочность (max_depth=8, criterion="entropy", splitter="random"):', accuracy)

Глубина дерева: 8 
Точность (max_depth=8, criterion="entropy", splitter="random"): 0.7716262975778547
CPU times: user 9.34 ms, sys: 1.86 ms, total: 11.2 ms
Wall time: 14 ms


Точность модели не изменилась при splitter="random".

### 3.2 Random Forest

- подберем гиперпараметр **__n_estimators__**

In [134]:
%%time

best_estim = 0
best_accuracy = 0

for estim in range(1, 101):
    model = RandomForestClassifier(random_state=2355555, n_estimators=estim)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(validation_data_features)
    accuracy = accuracy_score(validation_data_target, valid_predictions)
    # print('Количество деревьев', estim,'Точность', accuracy)
    if accuracy > best_accuracy:
        best_estim = estim
        best_accuracy = accuracy

print('Лучшая количество деревьев:', best_estim, 'Лучшая точность:', best_accuracy)     

Лучшая количество деревьев: 18 Лучшая точность: 0.7923875432525952
CPU times: user 32.2 s, sys: 909 ms, total: 33.1 s
Wall time: 40.2 s


Измерим точность модели при __n_estimators=18__.

In [135]:
%%time

best_forest_model = RandomForestClassifier(random_state=2355555, n_estimators=18)
best_forest_model.fit(train_data_features, train_data_target)
valid_predictions = best_forest_model.predict(validation_data_features)
accuracy = accuracy_score(validation_data_target, valid_predictions)
print('Лучшая количество деревьев: 18', 'Лучшая точность:', best_accuracy)   

Лучшая количество деревьев: 18 Лучшая точность: 0.7923875432525952
CPU times: user 158 ms, sys: 10.5 ms, total: 169 ms
Wall time: 197 ms


- ___criterion="gini" --> criterion="entropy"___

In [136]:
%%time

model = RandomForestClassifier(random_state=2355555, n_estimators=18, criterion='entropy')
model.fit(train_data_features, train_data_target)
valid_predictions = model.predict(validation_data_features)
accuracy = accuracy_score(validation_data_target, valid_predictions)
print('Лучшая количество деревьев: 18', 'Лучшая точность (criterion="entropy"):', best_accuracy)   

Лучшая количество деревьев: 18 Лучшая точность (criterion="entropy"): 0.7923875432525952
CPU times: user 179 ms, sys: 7.18 ms, total: 187 ms
Wall time: 193 ms


- **_bootstrap_ с True на False**

In [137]:
%%time

model = RandomForestClassifier(random_state=2355555, n_estimators=18, bootstrap=False)
model.fit(train_data_features, train_data_target)
valid_predictions = model.predict(validation_data_features)
accuracy = accuracy_score(validation_data_target, valid_predictions)
print('Лучшая количество деревьев: 18', 'Лучшая точность (bootstrap=False):', best_accuracy)  

Лучшая количество деревьев: 18 Лучшая точность (bootstrap=False): 0.7923875432525952
CPU times: user 161 ms, sys: 4.65 ms, total: 166 ms
Wall time: 170 ms


Никаких изменений в метрике accruracy нет.

- **_warm_start_ с False на True**

In [138]:
%%time

model = RandomForestClassifier(random_state=2355555, n_estimators=18, warm_start=True)
model.fit(train_data_features, train_data_target)
valid_predictions = model.predict(validation_data_features)
accuracy = accuracy_score(validation_data_target, valid_predictions)
print('Лучшая количество деревьев: 18', 'Лучшая точность (warm_start=False):', best_accuracy)  

Лучшая количество деревьев: 18 Лучшая точность (warm_start=False): 0.7923875432525952
CPU times: user 106 ms, sys: 5.12 ms, total: 111 ms
Wall time: 112 ms


Никаких изменений в метрике accruracy нет.

### 3.3 Logistic Regression

In [139]:
%%time

best_max_iter = 0
best_accuracy = 0

for max_iter in range(1, 201):
    model = LogisticRegression(random_state=2355555, max_iter=max_iter)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(validation_data_features)
    accuracy = accuracy_score(validation_data_target, valid_predictions)
    if accuracy > best_accuracy:
        best_max_iter = max_iter
        best_accuracy = accuracy

print('Лучшая количество итераций:', best_max_iter, '\nЛучшая точность:', best_accuracy)    

Лучшая количество итераций: 89 
Лучшая точность: 0.7197231833910035
CPU times: user 18.1 s, sys: 568 ms, total: 18.7 s
Wall time: 15.9 s


**Лучшее количество итераций = 89.**

In [140]:
%%time

best_reg_model = LogisticRegression(random_state=2355555)
best_reg_model.fit(train_data_features, train_data_target)
valid_predictions = best_reg_model.predict(validation_data_features)

accuracy = accuracy_score(validation_data_features, valid_predictions)
print('Количество итераций:', 100, '\nТочность:', accuracy)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets

Увеличение в большую сторону ничего не меняет. Изменим другой гиперпараметр solver.

In [141]:
%%time

best_solver = ''
best_accuracy = 0

for solve in {'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'}:
    model = LogisticRegression(random_state=2355555, solver=solve)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(validation_data_features)
    accuracy = accuracy_score(validation_data_target, valid_predictions)
    print('Solver:', solve, 'Точность:',accuracy)
    if accuracy > best_accuracy:
        best_solver= solve
        best_accuracy = accuracy
print()
print('Лучшая количество итерраций:', best_solver, '\nЛучшая точность:', best_accuracy)    

Solver: newton-cg Точность: 0.7197231833910035
Solver: sag Точность: 0.6643598615916955
Solver: lbfgs Точность: 0.7197231833910035
Solver: saga Точность: 0.6643598615916955
Solver: liblinear Точность: 0.698961937716263

Лучшая количество итерраций: newton-cg 
Лучшая точность: 0.7197231833910035
CPU times: user 665 ms, sys: 30.1 ms, total: 695 ms
Wall time: 1.11 s


**Изменим в цикле гиперпараметр _С_.**

In [142]:
%%time

best_C = 0.0
best_accuracy = 0

for float_c in [x/10 for x in range(1, 21)]:
    model = LogisticRegression(random_state=2355555, C=float_c)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(validation_data_features)
    accuracy = accuracy_score(validation_data_target, valid_predictions)
    if accuracy > best_accuracy:
        best_C= float_c
        best_accuracy = accuracy

print('Лучший C:', best_C,'\nЛучшая точность:', best_accuracy) 

Лучший C: 0.1 
Лучшая точность: 0.7197231833910035
CPU times: user 2.13 s, sys: 52.7 ms, total: 2.18 s
Wall time: 1.27 s


**ВЫВОД**

???

## 4 - Проверьте модель на тестовой выборке

In [143]:
test_predictions = best_tree_model.predict(test_data_features)
accuracy = accuracy_score(test_data_target, test_predictions)
print('Точность модели решающего леса на тестовой выборке:',accuracy)

test_predictions = best_forest_model.predict(test_data_features)
accuracy = accuracy_score(test_data_target, test_predictions)
print('Точность модели случайного леса на тестовой выборке:', accuracy)

test_predictions = best_reg_model.predict(test_data_features)
accuracy = accuracy_score(test_data_target, test_predictions)
print('Точность модели логистической регресиии на тестовой выборке:', accuracy)

Точность модели решающего леса на тестовой выборке: 0.7988165680473372
Точность модели случайного леса на тестовой выборке: 0.7943786982248521
Точность модели логистической регресиии на тестовой выборке: 0.764792899408284


**ВЫВОД**

- ?
- ?


## 5 - Проверьте модели на адекватность